In [5]:
import pandas as pd
import numpy as np
import folium
import json
# import geojson

In [6]:
# !pip install geojson
# !pip install folium

In [7]:
f = open('GeoJSON files/all_parks.geojson', 'r')
geojson = json.load(f)
# geojson

In [8]:
with open('GeoJSON files/westerpark_oost.geojson') as f:
    geo_westerparkoost = json.load(f)
    
with open('GeoJSON files/westerpark_centrum.geojson') as f:
    geo_westerparkcentrum = json.load(f) 
    
with open('GeoJSON files/westerpark_gasfabriek.geojson') as f:
    geo_westerparkgasfabriek = json.load(f) 
    
with open('GeoJSON files/westerpark_west.geojson') as f:
    geo_westerparkwest = json.load(f)
    
with open('GeoJSON files/erasmuspark.geojson') as f:
    geo_erasmuspark = json.load(f)
    
with open('GeoJSON files/oosterpark.geojson') as f:
    geo_oosterpark = json.load(f)
    
with open('GeoJSON files/vondelpark_west.geojson') as f:
    geo_vondelparkwest = json.load(f)
    
with open('GeoJSON files/vondelpark_oost3.geojson') as f:
    geo_vondelparkoost3 = json.load(f)

with open('GeoJSON files/vondelpark_oost2.geojson') as f:
    geo_vondelparkoost2 = json.load(f)
    
with open('GeoJSON files/vondelpark_oost1.geojson') as f:
    geo_vondelparkoost1 = json.load(f)
    
with open('GeoJSON files/sarphatipark.geojson') as f:
    geo_sarphatipark = json.load(f)
    
with open('GeoJSON files/rembrandtpark_noord.geojson') as f:
    geo_rembrandtparknoord = json.load(f)
    
with open('GeoJSON files/rembrandtpark_zuid.geojson') as f:
    geo_rembrandtparkzuid = json.load(f)
    
geos = [geo_rembrandtparknoord,
        geo_rembrandtparkzuid,
        geo_sarphatipark,
        geo_vondelparkwest,
        geo_vondelparkoost3,
        geo_vondelparkoost2,
        geo_vondelparkoost1,
        geo_oosterpark,
        geo_erasmuspark,
        geo_westerparkoost,
        geo_westerparkcentrum, 
        geo_westerparkgasfabriek,
        geo_westerparkwest]

In [9]:
locations = ['Rembrandtpark Noord',
 'Rembrandtpark Zuid',
 'Sarphatipark',
 'Vondelpark West',
 'Vondelpark Oost 3',
 'Vondelpark Oost 2',
 'Vondelpark Oost 1',
 'Oosterpark',
 'Erasmuspark',
 'Westerpark Oost',
 'Westerpark Centrum',
 'Westergasfabriek',
 'Westerpark West']


visits = [50, 20, 40, 60, 34, 90, 120, 23, 45, 78, 34, 13, 130]

data = pd.DataFrame(list(zip(locations, visits)),
               columns =['Location', 'Visitors'])
data

,Location,Visitors
0,Rembrandtpark Noord,50
1,Rembrandtpark Zuid,20
2,Sarphatipark,40
3,Vondelpark West,60
4,Vondelpark Oost 3,34
5,Vondelpark Oost 2,90
6,Vondelpark Oost 1,120
7,Oosterpark,23
8,Erasmuspark,45
9,Westerpark Oost,78


In [10]:
style_function = lambda x: {'fillColor': '#ffffff', 
                            'color':'#000000', 
                            'fillOpacity': 0.0, 
                            'weight': 0.5}
highlight_function = lambda x: {'fillColor': '#000000', 
                                'color':'#000000', 
                                'fillOpacity': 0.5, 
                                'weight': 0.5}

m = folium.Map(
    location=[52.377956, 4.897070],
    tiles="cartodbpositron",
    zoom_start=13,
)

chor = folium.Choropleth(
    geo_data=geojson,
    name="choropleth",
    data=data,
    columns=["Location", "Visitors"],
    key_on="properties.Naam",
    bins=[0, 25, 50, 75, 100, 125, 150],
    fill_color="YlOrRd",
    fill_opacity=0.8,
    line_opacity=0.7,
    legend_name="Visitors",
    highlight=True,
)

chor.add_to(m)

fgp = folium.FeatureGroup(name="Parks")

for g in geos:
    gjson = folium.GeoJson(g, 
                           name="City Parks",
                           style_function=style_function,
                           highlight_function=highlight_function,
                          )
    
    name = g['features'][0]['properties']['Naam']
    print(name)
    html=f'''
        <h1>{name}</h1><br>
        With a few lines of code...
        <p>
        <code>
            from numpy import *<br>
            exp(-2*pi)
        </code>
        </p>
        '''
    test = folium.Html(html, script=True)
    popup = folium.Popup(test, max_width=2650)
    gjson.add_child(popup)
    
    fgp.add_child(gjson)

fgp.add_to(m);
# folium.features.GeoJsonPopup(fields=['Naam'], labels=False).add_to(gjson)

Rembrandtpark Noord
Rembrandtpark Zuid
Sarphatipark
Vondelpark West
Vondelpark Oost 3
Vondelpark Oost 2
Vondelpark Oost 1
Oosterpark
Erasmuspark
Westerpark Oost
Westerpark Centrum
Westergasfabriek
Westerpark West


In [11]:
m